In [ ]:
#PCA - SelectKBest Samples used as reference:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
#from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

In [173]:
#def selectkbest(indep_X,dep_Y,n):
 #       test = SelectKBest(score_func=chi2, k=n)
  #      fit1= test.fit(indep_X,dep_Y)
   #     selectk_features = fit1.transform(indep_X)
    #    return selectk_features

    
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    

#def pca(indep_X,n):
        #pca = PCA(n_components = n)
        #X_train_pca = pca.fit_transform(X_train)
      #  X_test_pca = pca.transform(X_test)
        #explained_variance = pca.explained_variance_ratio_
        #return pca.explained_variance_ratio_, X_train_pca

def pca(X_train,X_test,n):
        pca = PCA(n_components = n)
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)
        explained_variance = pca.explained_variance_ratio_
        return explained_variance,X_train_pca,X_test_pca

def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
        
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
        #from sklearn.metrics import confusion_matrix
        #cm = confusion_matrix(y_test, y_pred)
        
     Accuracy=accuracy_score(y_test, y_pred )
        
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
   
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    
    
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski')
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return classifier,Accuracy,report,X_test,y_test,cm
def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      


def random(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [177]:
def PCA_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf): 
#def PCA_Classification(accknn,accnav,accdes,accrf): 
    dataframe=pd.DataFrame(index=['PCA'],columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random'])
   # dataframe=pd.DataFrame(index=['PCA'],columns=['KNN','Navie','Decision','Random'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex]=acclog[number]       
        dataframe['SVMl'][idex]=accsvml,[number]
        dataframe['SVMnl'][idex]=accsvmnl[number]
        dataframe['KNN'][idex]=accknn[number]
        dataframe['Navie'][idex]=accnav[number]
        dataframe['Decision'][idex]=accdes[number]
        dataframe['Random'][idex]=accrf[number]
    return dataframe

In [171]:
# Importing the dataset
dataset = pd.read_csv('Wine.csv')
indep_X = dataset.iloc[:, 0:13].values
dep_Y = dataset.iloc[:, 13].values

In [5]:
dataset.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


In [ ]:
dataset.columns

In [ ]:
dataset.shape

In [ ]:
dataset.notnull().sum()

In [217]:
X_train, X_test, y_train, y_test=split_scalar(indep_X,dep_Y)

In [218]:
#pca_best=pca(indep_X,dep_Y,7)       
ex_variance,train_pca,test_pca= pca(X_train,X_test,13)

acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]

In [219]:
#X_train, X_test, y_train, y_test=split_scalar(kbest,dep_Y)

#pca_best, X_train_pca = pca(indep_X,7)

classifier, Accuracy, report, X_test, y_test, cm = logistic(train_pca,y_train,test_pca)
acclog.append(Accuracy)

classifier,Accuracy,report,X_test,y_test,cm=svm_linear(train_pca,y_train,test_pca)  
accsvml.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=svm_NL(train_pca,y_train,test_pca)  
accsvmnl.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=knn(train_pca,y_train,test_pca)  
accknn.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Navie(train_pca,y_train,test_pca)  
accnav.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Decision(train_pca,y_train,test_pca)  
accdes.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=random(train_pca,y_train,test_pca)  
accrf.append(Accuracy)






In [220]:
#result=PCA_Classification(accknn,accnav,accdes,accrf)
result=PCA_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)

C:\Users\USER\AppData\Local\Temp\ipykernel_26040\3036049129.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataframe['Logistic'][idex]=acclog[number]
C:\Users\USER\AppData\Local\Temp\ipykernel_26040\3036049129.py:7: FutureWarning: Chain

In [188]:
result
#n=5

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
PCA,0.944444,"([0.9444444444444444, 0.9444444444444444, 0.94...",0.75,0.805556,0.944444,0.916667,0.888889


In [205]:
result
#n=7

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
PCA,1.0,"([0.9555555555555556], [0])",1.0,0.955556,1.0,0.933333,0.955556


In [210]:
result
#n=9

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
PCA,1.0,"([1.0], [0])",1.0,0.955556,1.0,0.955556,0.977778


In [216]:
result
#n=11

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
PCA,1.0,"([1.0], [0])",1.0,0.955556,1.0,0.933333,0.977778


In [221]:
result
#n=13

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
PCA,1.0,"([1.0], [0])",1.0,0.977778,1.0,0.955556,0.955556


In [198]:
feature_list = dataset.columns.tolist()
feature_list

['Alcohol',
 'Malic_Acid',
 'Ash',
 'Ash_Alcanity',
 'Magnesium',
 'Total_Phenols',
 'Flavanoids',
 'Nonflavanoid_Phenols',
 'Proanthocyanins',
 'Color_Intensity',
 'Hue',
 'OD280',
 'Proline',
 'Customer_Segment']

In [199]:
pca_feature_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
print(pca_feature_names)

['PC1', 'PC2', 'PC3', 'PC4', 'PC5']


In [222]:
X_pca

array([[-6.22149394e+01,  7.94217231e+00,  6.08669658e-01,
         1.44664622e+00,  1.27490647e+00],
       [-2.47627050e+02, -1.61320345e+01, -2.62328650e-01,
        -1.18730090e+00,  1.59944423e-02],
       [-3.97303052e+02,  4.20470704e+00, -3.88772466e+00,
        -2.26427104e-01, -8.93549769e-01],
       [ 7.68348344e+00,  1.89117232e+00,  4.28620532e+00,
         1.61131465e+00,  1.07534396e+00],
       [ 1.02587082e+02, -5.70689181e+00,  3.09685768e-01,
        -2.18208487e+00, -4.29858548e-01],
       [ 5.42597420e+02, -7.63625072e+00,  5.14246622e+00,
        -2.51401442e+00, -7.37895649e-01],
       [-4.52456427e+02, -4.92269745e+00, -4.19775554e+00,
        -7.67298592e-01, -2.67723067e-01],
       [-1.80499857e+02, -9.23954096e+00, -2.41762669e+00,
        -1.41622382e+00,  3.27905081e-01],
       [-3.08491393e+02, -7.16757824e+00, -5.87603729e-01,
        -1.63961850e+00, -1.19905599e+00],
       [ 1.02932964e+02,  1.42428196e+01, -3.95243840e+00,
         9.13977210e-01